In [18]:
import csv
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [19]:
def load_csv_to_trajectory(filepath):
    """Lädt die CSV und stellt Trajektorie-Liste wieder her"""
    trajectory = []
    with open(filepath, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe Kopfzeile

        for row in reader:
            numbers = list(map(float, row))
            trans = numbers[:3]
            rot = numbers[3:]
            trajectory.append((trans, rot))
    return trajectory

In [20]:
traj_1_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_1.csv'
traj_2_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_2.csv'
traj_3_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_3.csv'
traj_4_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_4.csv'

In [21]:
trajectory_1 = load_csv_to_trajectory(traj_1_path)
trajectory_2 = load_csv_to_trajectory(traj_2_path)
trajectory_3 = load_csv_to_trajectory(traj_3_path)
trajectory_4 = load_csv_to_trajectory(traj_4_path)

In [86]:
def compute_rectangle_corners(trajectory, width=3, height=4):
    """
    Berechnet die Rechteck-Eckpunkte (4 Stück) für jeden Trajektorienpunkt.

    :param trajectory: Liste von ([x, y, z], [roll, pitch, yaw]) Einträgen
    :param width: Breite des Rechtecks (in X-Richtung)
    :param height: Höhe des Rechtecks (in Y-Richtung)
    :return: Liste von Rechtecken, jedes als Liste von 4 Eckpunkten (jeweils [x, y, z])
    """
    rectangles = []

    # Rechteck im lokalen TCP-Koordinatensystem (XY-Ebene, Zentrum im Ursprung)
    half_w = width / 2
    half_h = height / 2
    local_corners = np.array([
        [-half_w, 0, -half_h],
        [ half_w, 0, -half_h],
        [ half_w, 0, half_h],
        [-half_w, 0, half_h]
    ])

    for trans, rpy in trajectory:
        # RPY in Radiant, falls nicht schon so
        rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(trans)
        rectangles.append(translated_corners.tolist())
    return rectangles

In [ ]:
rectangles_1 = compute_rectangle_corners(trajectory_1)
rectangles_2 = compute_rectangle_corners(trajectory_2)
rectangles_3 = compute_rectangle_corners(trajectory_3)
rectangles_4 = compute_rectangle_corners(trajectory_4)

In [156]:
def visualize_rectangles(rectangles, figsize=(6, 6), connect_points=None, 
                         secondary_rectangles=None, secondary_legend_title=None):
    """
    Visualisiert Rechtecke + Trajektorie im 3D-Raum mit Legende und Bezeichnungen der Trajektorienpunkte.
    
    Zusätzlich kann ein zweiter Satz Rechtecke (grau & transparent) dargestellt werden – inkl. Mittelpunktslinie.

    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
    :param figsize: Größe der Plotfläche (Breite, Höhe)
    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
    :param secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)
    :param secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke
    """
    %matplotlib qt

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    import numpy as np
    import matplotlib.cm as cm

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')

    centers = []
    cmap = cm.get_cmap('tab20')
    legend_handles = []

    # === Hauptrechtecke (farbig) ===
    for i, rect in enumerate(rectangles):
        color = cmap(i / len(rectangles))
        poly = Poly3DCollection([rect], facecolors=color, alpha=0.2, edgecolors=color)
        ax.add_collection3d(poly)

        if i == 0:
            label = 'Pre-Position'
        elif i == 1:
            label = 'Spitze'
        elif i == len(rectangles) - 1:
            label = 'Loslassen'
        else:
            label = str(i + 1)

        legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=label,
                                         markersize=10, markerfacecolor=color))

        center = np.mean(np.array(rect), axis=0)
        centers.append(center)
        ax.scatter(*center, color='red', s=30)

    # Linie durch die Haupt-Mittelpunkte
    centers = np.array(centers)
    ax.plot(centers[:, 0], centers[:, 1], centers[:, 2], color='black', linewidth=2)

    # Beschriftung der Hauptlinie
    if len(centers) > 0:
        ax.text(*centers[0], 'Pre-Position', color='black', fontsize=10)
    if len(centers) > 2:
        ax.text(*centers[-1], 'Loslassen', color='black', fontsize=10)

    # === Sekundärrechtecke (grau) ===
    if secondary_rectangles is not None:
        secondary_centers = []
        for rect in secondary_rectangles:
            poly = Poly3DCollection([rect], facecolors='green', alpha=0.2, edgecolors='gray')
            ax.add_collection3d(poly)

            center = np.mean(np.array(rect), axis=0)
            secondary_centers.append(center)
            ax.scatter(*center, color='green', s=20)

        # Linie durch graue Mittelpunkte
        secondary_centers = np.array(secondary_centers)
        ax.plot(secondary_centers[:, 0], secondary_centers[:, 1], secondary_centers[:, 2],
                color='green', linewidth=1, linestyle='--')

        # Legendenhandle für Sekundärrechtecke
        if secondary_legend_title:
            legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=secondary_legend_title,
                                             markersize=10, markerfacecolor='green'))

    # === Verbindungslinie (optional) ===
    if connect_points is not None and len(connect_points) == 2:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]],
                color='gray', linestyle='dashed', linewidth=2)
        ax.scatter(*p1, 'o', color='magenta', linewidth=6)
        ax.scatter(*p2, 'o', color='magenta', linewidth=6)
        ax.text(*p1, 'Spitze', color='magenta', fontsize=12, ha='center')
        ax.text(*p2, 'Senke', color='magenta', fontsize=12, ha='center')

    # === Achseneinstellungen ===
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Einfädelung Trajektorie Spitze -> Senke (TCP in WORLD-Frame)')

    all_points = rectangles + (secondary_rectangles if secondary_rectangles else [])
    all_points = np.vstack(all_points)
    mid = np.mean(all_points, axis=0)
    max_range = np.ptp(all_points, axis=0).max() / 2
    for i, axis in enumerate([ax.set_xlim, ax.set_ylim, ax.set_zlim]):
        axis(mid[i] - max_range, mid[i] + max_range)

    ax.set_box_aspect([1, 1, 1])
    ax.legend(handles=legend_handles, title="Trajektorien-Punkte", loc='upper left')
    plt.tight_layout(pad=0.5, w_pad=0.5, h_pad=0.5)
    plt.show()

In [157]:
visualize_rectangles(rectangles = rectangles_1,
                     secondary_rectangles = rectangles_3,
                     secondary_legend_title = 'Trajektorie 3',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))

visualize_rectangles(rectangles = rectangles_2,
                     secondary_rectangles = rectangles_3,
                     secondary_legend_title = 'Trajektorie 3',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))

visualize_rectangles(rectangles = rectangles_3,
                     secondary_rectangles = rectangles_3,
                     secondary_legend_title = 'Trajektorie 3',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))

visualize_rectangles(rectangles = rectangles_4,
                     secondary_rectangles = rectangles_3,
                     secondary_legend_title = 'Trajektorie 3',
                     connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))

/tmp/ipykernel_118458/1716797752.py:25: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20')
